**Note: be sure to add the folder Brown-Datathon-Public-2020 to your drive (you can do this using the dropdown arrow next to the folder in the path header in drive) before running this notebook.**

## Presets

In [0]:
# Make necessary/useful imports here
import pandas as pd
import random
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.cm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Set pandas options to not truncate
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# # Do not mount data!! 
# #from google.colab import drive
# #drive.mount("/content/gdrive", force_remount=True)
# # Instead, download and unzip data
!wget https://datathon2020.s3.amazonaws.com/datasets/citizens-home-financing-challenge.zip
!unzip citizens-home-financing-challenge.zip 



--2020-02-23 16:03:16--  https://datathon2020.s3.amazonaws.com/datasets/citizens-home-financing-challenge.zip
Resolving datathon2020.s3.amazonaws.com (datathon2020.s3.amazonaws.com)... 52.216.96.163
Connecting to datathon2020.s3.amazonaws.com (datathon2020.s3.amazonaws.com)|52.216.96.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2972199826 (2.8G) [application/zip]
Saving to: ‘citizens-home-financing-challenge.zip’

citizens-home-finan 100%[===================>]   2.77G  88.5MB/s    in 45s     

2020-02-23 16:04:01 (63.3 MB/s) - ‘citizens-home-financing-challenge.zip’ saved [2972199826/2972199826]

Archive:  citizens-home-financing-challenge.zip
   creating: citizens-home-financing-challenge/
   creating: citizens-home-financing-challenge/dictionaries/
  inflating: citizens-home-financing-challenge/dictionaries/datathon_summary.csv  
  inflating: citizens-home-financing-challenge/dictionaries/zip9_coded_20190X_dict.csv  
  inflating: citizens-home-financi

In [0]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False, handle_unknown = 'ignore')
mmscaler = MinMaxScaler()
sscaler = StandardScaler()
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score, accuracy_score, roc_curve, auc
from sklearn.feature_selection import RFE # for feature reduction
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

## Data Dictionaries

## Looking at Data Samples

In [0]:
n = 6009259 # number of records in file
s = 756666 # sample size (can/should be changed to your preference)

root_path = 'citizens-home-financing-challenge/'

skip_list = sorted(random.sample(range(1,n+1),n-s))
apr_df = pd.read_csv(root_path + "zip9_coded_201904_pv.csv", skiprows=skip_list, dtype={'zip5': str})
may_df = pd.read_csv(root_path + "zip9_coded_201905_pv.csv", skiprows=skip_list, dtype={'zip5': str})
jun_df = pd.read_csv(root_path + "zip9_coded_201906_pv.csv", skiprows=skip_list, dtype={'zip5': str})
jul_df = pd.read_csv(root_path + "zip9_coded_201907_pv.csv", skiprows=skip_list, dtype={'zip5': str})
aug_df = pd.read_csv(root_path + "zip9_coded_201908_pv.csv", skiprows=skip_list, dtype={'zip5': str})
sep_df = pd.read_csv(root_path + "zip9_coded_201909_pv.csv", skiprows=skip_list, dtype={'zip5': str})
demo_df = pd.read_csv(root_path + "zip9_demographics_coded_pv.csv", skiprows=skip_list, dtype={'zip5': str})

In [0]:
## Count non-missing
def count_nonmissing(data, column_list, new_column):  
  result = np.zeros(data.shape[0])  
  for col_name in column_list:
    i = 0
    for BOOLEAN in pd.notnull(data[col_name]):
      if BOOLEAN:
        result[i] += 1
        i += 1  
  data[new_column] = result

In [0]:
def create_variables(data,column_list_1, new_column_1,column_list_2, new_column_2, all_column):  
  data = data.copy()
  count_nonmissing(data,column_list_1, new_column_1)  
  count_nonmissing(data,column_list_2, new_column_2)  
  data['mean_mortgage_balance']=data['total_mortgage_balance']/data['mortgage_count']  
  data['mean_mortgage_limit']=data['total_mortgage_limit']/data['mortgage_count']  
  data['mean_mortgage_trades']=data['total_mortgage_trades']/data['mortgage_count']  
  data['mean_homeequity_balance']=data['total_homeequity_balance']/data['homeequity_count']  
  data['mean_homeequity_limit']=data['total_homeequity_limit']/data['homeequity_count']  
  data['mean_homeequity_trades']=data['total_homeequity_trades']/data['homeequity_count']  
  data=data[all_column]
  data = data.replace([np.inf, -np.inf], np.nan)
  data = data.fillna(0)
  
  return data

In [0]:
column_list_1=['mortgage1_limit','mortgage2_limit', 'mortgage3_limit','mortgage4_limit','mortgage5_limit']
column_list_2=['homeequity1_limit','homeequity2_limit','homeequity3_limit','homeequity4_limit','homeequity5_limit']
all_column=['zip5', 'zip9_code', 'bankcard_limit', 'bankcard_balance',      
            'bankcard_trades', 'bankcard_util', 'total_revolving_limit',     
            'total_revolving_balance', 'total_revolving_trades',     
            'total_revolving_util','total_mortgage_limit', 'total_mortgage_balance',      
            'total_mortgage_trades', 'mortgage1_loan_to_value', 'total_homeequity_limit',     
            'total_homeequity_balance', 'total_homeequity_trades',      
            'homeequity1_loan_to_value', 'autoloan_open', 'studentloan_open',     
            'bankcard_open', 'homeequity_open', 'mortgage1_open', 'mortgage2_open','mortgage3_open', 
            'mortgage4_open', 'mortgage5_open', 'mortgage_open', 'mortgage_count',     
            'mean_mortgage_balance', 'mean_mortgage_limit', 'homeequity_count',      
            'mean_homeequity_balance', 'mean_homeequity_limit',     
            'mean_mortgage_trades', 'mean_homeequity_trades']

In [0]:
## Preprocess with each month data
apr_df0 = create_variables(apr_df,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
may_df0 = create_variables(may_df,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
jun_df0 = create_variables(jun_df,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
jul_df0 = create_variables(jul_df,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
aug_df0 = create_variables(aug_df,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
sep_df0 = create_variables(sep_df,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)

**Forming final dataset to fit the model**

In [0]:
from functools import reduce
dfs = [apr_df0, may_df0, jun_df0, jul_df0, aug_df0, sep_df0]
# df_final = reduce(lambda left, right: pd.merge(left,right, on = ['zip5', 'zip9_code'], how='outer'), dfs)

In [0]:
months = range(4,10)
df_final = pd.DataFrame()
for i in range(6):
  for col_name in dfs[i].columns:
    if "zip" not in col_name:
          df_final[col_name + "_"+ str(i+4)]  = dfs[i][col_name]
    else:
      if i == 0:
        df_final[col_name]  = dfs[i][col_name]

In [0]:
# Merge monthly data with demo
final_demo_merge = df_final.merge(demo_df,
                              how='inner',
                              on= ['zip9_code', 'zip5'],
                              validate='one_to_one')

In [0]:
# feature engineering: adding indicator features
six_month_buy = final_demo_merge['mortgage_count_9'] - final_demo_merge['mortgage_count_4']
six_month_first_buy = final_demo_merge['mortgage_open_9'] - final_demo_merge['mortgage_open_4']
six_month_first_buy_ind = pd.DataFrame([1 if i > 0 else 0 for i in six_month_first_buy])
final_demo_merge['six_month_buy'] = six_month_buy
final_demo_merge['six_month_first_buy_ind'] = six_month_first_buy_ind

In [0]:
# feature engineering: adding new response variables
num_zip9_in_zip5 = final_demo_merge.groupby("zip5").agg({'zip9_code' : ['count']})

In [0]:
zip5_firstbuy = final_demo_merge.groupby("zip5").agg({'first_homebuyers' : ['sum']})
zip5_firstbuy = pd.DataFrame(zip5_firstbuy).reset_index()
zip5_firstbuy.columns = ['zip5', 'zip5_firstbuy']
# zip5_firstbuy.head()

In [0]:
zip5_homebuy = final_demo_merge.groupby("zip5").agg({'homebuyers' : ['sum']})
zip5_homebuy = pd.DataFrame(zip5_homebuy).reset_index()
zip5_homebuy.columns = ['zip5', 'zip5_homebuy']
# zip5_homebuy.head()

In [0]:
## Merge with new response variables
final_demo_merge1 = final_demo_merge.merge(zip5_homebuy,
                              how='inner',
                              on= ['zip5'],
                              validate='many_to_one').merge(zip5_firstbuy,
                              how='inner',
                              on= ['zip5'],
                              validate='many_to_one')

In [0]:
final_demo_merge1['homebuy_prop_zip9_in_zip5'] = final_demo_merge1['homebuyers'] / final_demo_merge1['zip5_homebuy']
final_demo_merge1['firstbuy_prop_zip9_in_zip5'] = final_demo_merge1['first_homebuyers'] / final_demo_merge1['zip5_firstbuy']

In [0]:
final_demo_merge1.drop(['zip5_homebuy', 'zip5_firstbuy'], axis = 1, inplace=True)

In [0]:
## Final Dataset
final_demo_merge1 = final_demo_merge1.fillna(0)

**Train the Random Forest Regression model on proportions of homebuy and firstbuy for each zip9 area in a zip5 code**

In [0]:
y_names = ['homebuy_prop_zip9_in_zip5', 'firstbuy_prop_zip9_in_zip5']

# buyer 
y1 = final_demo_merge1[y_names[0]]

# first 
y2 = final_demo_merge1[y_names[1]]

X = final_demo_merge1.drop(columns = y_names)

In [21]:
## Train Random Forests Model with 72,000 instance to tune the hyperparameters
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupShuffleSplit

random_seed = 1030
gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=random_seed)
gss.get_n_splits()
for train_idx, test_idx in gss.split(X, y1, X["zip5"]):
  print("TRAIN:", train_idx, "TEST:", test_idx)

X_other = pd.DataFrame(X.iloc[train_idx, :][0:60000])
X_test =  pd.DataFrame(X.iloc[test_idx, :][0:10000])
y1_other = pd.DataFrame(np.array(y1.iloc[train_idx][0:60000]).reshape(-1, 1))
y2_other = pd.DataFrame(np.array(y2.iloc[train_idx][0:60000]).reshape(-1, 1))
y1_test = pd.DataFrame(np.array(y1.iloc[test_idx][0:10000]).reshape(-1, 1))
y2_test = pd.DataFrame(np.array(y2.iloc[test_idx][0:10000]).reshape(-1, 1))


TRAIN: [     0      1      2 ... 756663 756664 756665] TEST: [   103    104    105 ... 756291 756292 756571]


In [0]:
from sklearn.model_selection import GroupKFold
def ML_pipeline_stratify_kfold_rf(X_other, y1_other, y2_other, X_test, y1_test, y2_test, random_state,n_folds):
 
    gkf = GroupKFold(n_splits=n_folds)
    gkf.get_n_splits(X_other, y1_other, X_other['zip5'])
    
    # You'll need to loop through the 5 options the CV fold can be selected
    for train_index, CV_index in gkf.split(X_other, y1_other, X_other['zip5']):
        X_train, X_CV = X_other.iloc[train_index], X_other.iloc[CV_index]
        y1_train, y1_CV = y1_other.iloc[train_index], y1_other.iloc[CV_index]
        y2_train, y2_CV = y2_other.iloc[train_index], y2_other.iloc[CV_index]

        print(X_train.shape, X_CV.shape)
        print(y1_train.shape, y1_CV.shape)
        print(y2_train.shape, y2_CV.shape)
        # tune rf hyper-parameters, max_depth and min_samples_split
        data1 = []
        data2 = []
        # CV_scores1 = []
        # CV_scores2 = []
        rf1_test_scores = []
        rf2_test_scores = []
        for i in range(2,6):
            for j in range(2,6):
                clf1 = RandomForestRegressor(n_estimators= 100,max_depth=i, min_samples_split = j,random_state=random_state)
                clf2 = RandomForestRegressor(n_estimators= 100,max_depth=i, min_samples_split = j,random_state=random_state)
                clf1.fit(X_train,y1_train)
                clf2.fit(X_train,y2_train)
                data1.append([i,j, clf1, clf1.score(X_CV,y1_CV)])
                data2.append([i,j, clf2, clf2.score(X_CV,y1_CV)])

    df1 = pd.DataFrame(data1, columns = ['max_depth', 'min_samples_split', 'model', 'cv_score'])
    df2 = pd.DataFrame(data2, columns = ['max_depth', 'min_samples_split', 'model', 'cv_score'])
    cv_best1 = max(df1['cv_score'])
    cv_best2 = max(df2['cv_score'])
    index1 = np.argmax(np.array(df1['cv_score']))
    index2 = np.argmax(np.array(df2['cv_score']))
    rf1_best_pair = (df1['max_depth'][index1], df1['min_samples_split'][index1])
    rf2_best_pair = (df2['max_depth'][index2], df2['min_samples_split'][index2])
    rf1 = df1['model'][index1]
    rf2 = df2['model'][index2]
    # CV_scores1.append(cv_best1)
    # CV_scores2.append(cv_best2)
    rf1_predicted = rf1.predict(X_test)
    rf2_predicted = rf2.predict(X_test)
    rf1_test_scores.append(clf1.score(X_test, y1_test))
    rf2_test_scores.append(clf2.score(X_test, y2_test))
        
    return rf1_test_scores, rf1_best_pair, y1_test, rf1_predicted, rf1, rf2_test_scores, rf2_best_pair, y2_test, rf2_predicted, rf2

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
random_state = 1030
n_folds = 2
rf1_test_scores, rf1_best_pair, y1_test, rf1_predicted, rf1, rf2_test_scores, rf2_best_pair, y2_test, rf2_predicted, rf2 = ML_pipeline_stratify_kfold_rf(X_other, y1_other, y2_other, X_test, y1_test, y2_test,random_state,n_folds)

(30000, 213) (30000, 213)
(30000, 1) (30000, 1)
(30000, 1) (30000, 1)
(30000, 213) (30000, 213)
(30000, 1) (30000, 1)
(30000, 1) (30000, 1)


In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

## Model performance
print("MAE of buying home:", mean_absolute_error(y1_test, rf1_predicted))
print("MAE of buying first home:", mean_absolute_error(y2_test, rf2_predicted))
print("R2 of buying home:", r2_score(y1_test, rf1_predicted))
print("R2 of buying first home:", r2_score(y2_test, rf2_predicted))
print("RMSE of buying home:", np.sqrt(mean_squared_error(y1_test, rf1_predicted)))
print("RMSE of buying first home:", np.sqrt(mean_squared_error(y2_test, rf2_predicted)))
print("best depth and min_split pair of rf1:", rf1_best_pair, "||test r-square:", rf1_test_scores)
print("best depth and min_split pair of rf2:", rf2_best_pair, "||test r-square:", rf2_test_scores)

MAE of buying home: 0.009140802173638964
MAE of buying first home: 0.006537338698637083
R2 of buying home: 0.6127258084475449
R2 of buying first home: 0.6999396412000312
RMSE of buying home: 0.059101670231173145
RMSE of buying first home: 0.04955145063684216
best depth and min_split pair of rf1: (5, 5) ||test r-square: [0.6127258084475449]
best depth and min_split pair of rf2: (2, 2) ||test r-square: [0.7040322353267381]


In [29]:
rf1 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1030, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1030, verbose=0, warm_start=False)

In [30]:
rf2 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=2, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1030, verbose=0, warm_start=False)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=2, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1030, verbose=0, warm_start=False)

**Re-train the best tuned RF model on 750,000 instances**

In [23]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
random_seed = 1030
gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=random_seed)
gss.get_n_splits()
for train_idx, test_idx in gss.split(X, y1, X["zip5"]):
  print("TRAIN:", train_idx, "TEST:", test_idx)

X_train = X.iloc[train_idx,:]
X_test =  X.iloc[test_idx, :]
y1_train = pd.DataFrame(np.array(y1.iloc[train_idx]).reshape(-1, 1))
y2_train = pd.DataFrame(np.array(y2.iloc[train_idx]).reshape(-1, 1))
y1_test = pd.DataFrame(np.array(y1.iloc[test_idx]).reshape(-1, 1))
y2_test = pd.DataFrame(np.array(y2.iloc[test_idx]).reshape(-1, 1))

TRAIN: [     0      1      2 ... 756663 756664 756665] TEST: [   114    168    169 ... 756296 756547 756605]


In [0]:
print(X_train.shape, X_test.shape)
print(y1_train.shape, y1_test.shape)
print(y2_train.shape, y2_test.shape)

In [27]:
rf1.fit(X_train, y1_train)
rf2.fit(X_train, y2_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=2, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1030, verbose=0, warm_start=False)

In [0]:
y1_predict = rf1.predict(X_test)
y2_predict = rf2.predict(X_test)

In [0]:
zip5 = []
zip9 = []
for i in X_test['zip5']:
  zip5.append(str(i))

for i in X_test['zip9_code']:
  zip9.append(str(i))

result_rf = pd.DataFrame()

result_rf['proportion of buyers'] = y1_predict
result_rf['proportion of first time buyers'] = y2_predict
result_rf['zip5'] = pd.DataFrame(zip5)
result_rf['zip9'] = pd.DataFrame(zip9)
result_rf.to_csv('rf.csv')

In [32]:
print("MAE of buying home:", mean_absolute_error(y1_test, y1_predict))
print("MAE of buying first home:", mean_absolute_error(y2_test, y2_predict))
print("R2 of buying home:", r2_score(y1_test, y1_predict))
print("R2 of buying first home:", r2_score(y2_test, y2_predict))
print("RMSE of buying home:", np.sqrt(mean_squared_error(y1_test, y1_predict)))
print("RMSE of buying first home:", np.sqrt(mean_squared_error(y2_test, y2_predict)))

MAE of buying home: 0.006984064547492014
MAE of buying first home: 0.005113626088252926
R2 of buying home: 0.6204266993396539
R2 of buying first home: 0.6988230317650438
RMSE of buying home: 0.0505489333585644
RMSE of buying first home: 0.04381113028240972


In [0]:
def loss_function_2(y_pred, y_labels):
    size = y_pred.size
    differences = y_pred - y_labels
    
    # If the prediction is smaller than the label, then we have underpredicted, scale this up
    under = np.where(differences < 0, differences, 0)
    over = np.where(differences >= 0, differences, 0)
    under_square_scaled = under*under*10
    over_square = over*over
    square_sum = np.sum(under_square_scaled) + np.sum(over_square)
    
    return square_sum/size

In [34]:
## Weight loss for RF model
y1_wl_rf = loss_function_2(pd.DataFrame(np.array(y1_predict).reshape(-1, 1)), y1_test)
y2_wl_rf = loss_function_2(pd.DataFrame(np.array(y2_predict).reshape(-1, 1)), y2_test)
print(y1_wl_rf, y2_wl_rf)
print((y1_wl_rf+y2_wl_rf)/2)

0.01915601924618983 0.013211098415803842
0.016183558830996835


In [0]:
# classified_data = pd.DataFrame()

# classified_data['pred_1'] = (y1_predict != 0) + 0
# classified_data['pred_2'] = (y2_predict != 0) + 0
# classified_data['true_1'] = (y1_test != 0) + 0
# classified_data['true_2'] = (y2_test != 0) + 0
# classified_data['pred_1_regr'] = y1_predict
# classified_data['pred_2_regr'] = y2_predict
# classified_data['true_1_regr'] = y1_test
# classified_data['true_2_regr'] = y2_test

**Train the XGBoost Regression model on proportions of homebuy and firstbuy for each zip9 area in a zip5 code**

**We have tuned the hyperparameters for our XGBoost model on the other notebook with grid search gourpfold k-folder cross vaildation, use scoring = 'neg_mean_squared_error'. It turned out that the best hyperparapeter pairs are {eta:0.01, gamma:0, max_depth:2}. So we are going to use this set to fit 750,000 instances.**

In [0]:
import xgboost as xgb
xgbr1 = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.001, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs= -1, nthread=None, objective='reg:linear', random_state=1030,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

xgbr2 = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.001, gamma=0.001,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=166,
             n_jobs= -1, nthread=None, objective='reg:linear', random_state=1030,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [25]:
x_train_xgbr = X_train.drop('zip5',axis=1)
xgbr1.fit(x_train_xgbr, y1_train)
xgbr2.fit(x_train_xgbr, y2_train)

[15:30:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.001, gamma=0.001,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=166,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=1030,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
x_test_xgbr = X_test.drop('zip5',axis=1)

y1_predict_xgb = xgbr1.predict(x_test_xgbr)
y2_predict_xgb = xgbr2.predict(x_test_xgbr)

In [38]:
print("MAE of buying home:", mean_absolute_error(y1_test, y1_predict_xgb))
print("MAE of buying first home:", mean_absolute_error(y2_test, y2_predict_xgb))
print("R2 of buying home:", r2_score(y1_test, y1_predict_xgb))
print("R2 of buying first home:", r2_score(y2_test, y2_predict_xgb))
print("RMSE of buying home:", np.sqrt(mean_squared_error(y1_test, y1_predict_xgb)))
print("RMSE of buying first home:", np.sqrt(mean_squared_error(y2_test, y2_predict_xgb)))

MAE of buying home: 0.007280715861502897
MAE of buying first home: 0.005214192609207299
R2 of buying home: 0.6171824889369386
R2 of buying first home: 0.7100294408258285
RMSE of buying home: 0.050764494432524766
RMSE of buying first home: 0.04298832584459643


In [39]:
## Weight loss for XGB Model
y1_wl_xgb = loss_function_2(pd.DataFrame(np.array(y1_predict_xgb).reshape(-1, 1)), y1_test)
y2_wl_xgb = loss_function_2(pd.DataFrame(np.array(y2_predict_xgb).reshape(-1, 1)), y2_test)
print(y1_wl_xgb, y2_wl_xgb)
print((y1_wl_xgb+y2_wl_xgb)/2)

0.019434059508015664 0.012688221921094455
0.01606114071455506


In [0]:
result_xgb = pd.DataFrame()

result_xgb['proportion of buyers'] = y1_predict_xgb
result_xgb['proportion of first time buyers'] = y2_predict_xgb
result_xgb['zip5'] = pd.DataFrame(zip5)
result_xgb['zip9'] = pd.DataFrame(zip9)
result_xgb.to_csv('xgb.csv')

In [0]:
import pandas as pd


**Test set prediction**

In [9]:
root_path = 'citizens-home-financing-challenge/'

apr_df11 = pd.read_csv(root_path + "zip9_coded_201904_wh.csv")
may_df11 = pd.read_csv(root_path + "zip9_coded_201905_wh.csv")
jun_df11 = pd.read_csv(root_path + "zip9_coded_201906_wh.csv")
jul_df11 = pd.read_csv(root_path + "zip9_coded_201907_wh.csv")
aug_df11 = pd.read_csv(root_path + "zip9_coded_201908_wh.csv")
sep_df11 = pd.read_csv(root_path + "zip9_coded_201909_wh.csv")
demo_df1 = pd.read_csv(root_path + "zip9_demographics_unlabeled_wh_test.csv")

FileNotFoundError: ignored

In [4]:
apr_df1 = create_variables(apr_df11,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
may_df1 = create_variables(may_df11,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
jun_df1 = create_variables(jun_df11,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
jul_df1 = create_variables(jul_df11,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
aug_df1 = create_variables(aug_df11,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)
sep_df1 = create_variables(sep_df11,column_list_1,'mortgage_count',column_list_2,'homeequity_count', all_column)

NameError: ignored

In [3]:
jun_df1.shape

NameError: ignored

In [4]:
dfs1 = [apr_df1, may_df1, jun_df1, jul_df1, aug_df1, sep_df1]

NameError: ignored

In [5]:
months = range(4,10)
df_final1 = pd.DataFrame()
for i in range(6):
  for col_name in dfs1[i].columns:
    if "zip" not in col_name:
          df_final1[col_name + "_"+ str(i+4)]  = dfs1[i][col_name]
    else:
      if i == 0:
        df_final1[col_name]  = dfs1[i][col_name]

NameError: ignored

In [65]:
df_final1.shape

(667418, 206)

In [0]:
# Merge monthly data with demo
final_demo_merge2 = df_final1.merge(demo_df1,
                              how='inner',
                              on= ['zip9_code'],
                              validate='one_to_one')




In [41]:
demo_df1.head()

,zip9_code,age,household_count,person_count
0,3797056,44.246817,6,12
1,3068313,37.451611,8,21
2,3203991,51.953018,6,7
3,324866,46.804237,5,9
4,2899974,54.523189,5,10


In [0]:
agg = pd.read_csv(root_path + "agg.csv")
agg.columns = ['Unnamed: 0', 'zip9_code', 'homebuyers', 'first_homebuyers']
agg.drop('Unnamed: 0', axis = 1, inplace = True)

In [0]:
final_demo_merge2 = final_demo_merge2.merge(agg,
                              how='inner',
                              on= ['zip9_code'],
                              validate='one_to_one')

In [94]:
final_demo_merge2.shape

(667418, 215)

In [0]:
six_month_buy1 = final_demo_merge2['mortgage_count_9'] - final_demo_merge2['mortgage_count_4']
six_month_first_buy1 = final_demo_merge2['mortgage_open_9'] - final_demo_merge2['mortgage_open_4']
six_month_first_buy_ind1 = pd.DataFrame([1 if i > 0 else 0 for i in six_month_first_buy1])
final_demo_merge2['six_month_buy'] = six_month_buy1
final_demo_merge2['six_month_first_buy_ind'] = six_month_first_buy_ind1

In [96]:
final_demo_merge2.shape

(667418, 215)

In [35]:
X = final_demo_merge2.drop('zip5', axis = 1)
X.columns

Index(['zip9_code', 'bankcard_limit_4', 'bankcard_balance_4',
       'bankcard_trades_4', 'bankcard_util_4', 'total_revolving_limit_4',
       'total_revolving_balance_4', 'total_revolving_trades_4',
       'total_revolving_util_4', 'total_mortgage_limit_4',
       ...
       'mean_homeequity_limit_9', 'mean_mortgage_trades_9',
       'mean_homeequity_trades_9', 'age', 'household_count', 'person_count',
       'homebuyers', 'first_homebuyers', 'six_month_buy',
       'six_month_first_buy_ind'],
      dtype='object', length=212)

In [0]:
y_te1 = xgbr1.predict(X)
y_te2 = xgbr2.predict(X)

In [0]:

zip9 = []

for i in X['zip9_code']:
  zip9.append(str(i))

In [0]:
result_xgb = pd.DataFrame()

result_xgb['proportion of buyers'] = y_te1
result_xgb['proportion of first time buyers'] = y_te2

result_xgb['zip9'] = pd.DataFrame(zip9)
result_xgb.to_csv('result.csv')

148892

In [0]:
X['zip9_code'].head()